In [ ]:
# 当 huge increase/decrease 时，if volume is huge, means争议很大, then 设置一个止损 止盈， like -25% vs +75% for a huge drop， compare probability, idealily 50% vs 50%

In [1]:
import multiprocessing
import itertools
import requests
import pandas as pd
from polygon import RESTClient
import pandas as pd
from polygonAPIkey import polygonAPIkey
from pandas_datareader import data as pdr
from tqdm.auto import tqdm
import multiprocessing as mp
import numpy as np
import yfinance as yf
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

yf.pdr_override()

In [1]:
# 用historical std来做trading rule, and also stop win and stop loss rule, this way you avoid overfitting the threshold value by selecting biasly
# 但是hand seelct the std threshold is kind of unavoidable
# volume也可以用historical std

# gradually increase of volatilitys 开始介入
# decrease of volatilitys 退出
# 仓位时间可以ml吗

    

In [2]:
# def rel_change(df, lookback_period_days, column_name):
#     df[column_name + '_rel_change'] = 0
#     for i in range(lookback_period_days, len(df)):
#         historical = df.iloc[i-lookback_period_days:i][column_name]
#         today = df.iloc[i][column_name]
#         df.loc[df.index[i], column_name + '_rel_change'] = (today - historical.mean()) / historical.mean()
#     return df

def feature_gen(tick):
    prices = pdr.get_data_yahoo(tick)

    prices['pct_ret'] = prices['Adj Close'].pct_change()
    prices['avg_pct_ret'] = prices['pct_ret'].expanding().mean()
    prices['std_pct_ret'] = prices['pct_ret'].expanding().std()
    
    prices['avg_vol'] = prices['Volume'].expanding().mean()
    prices['std_vol'] = prices['Volume'].expanding().std()
    
    prices['ret_t_score'] = (prices['pct_ret'] - prices['avg_pct_ret']) / prices['std_pct_ret']
    
    prices['vol_t_score'] = (prices['Volume'] - prices['avg_vol']) / prices['std_vol']
    
    
    # y
    #找之后n天的high low, examine if related
    # how to decide n though?
    
    return prices[['ret_t_score', 'vol_t_score', 'pct_ret']]
    
    

    # prices = prices[5:]
    # # Averaging high change and low change, so capture volatility
    # prices['Avg_rel_change'] = (prices['Adj High_rel_change'] - prices['Adj Low_rel_change']) / 2
    
    # data = prices.loc[:, ['Adj Close', 'Volume', 'Volume_rel_change', 'Avg_rel_change']]
    # data['transaction amount'] = data['Adj Close'] * data['Volume']
    # # Y is next day adjusted close price relative change
    # data['pct_ret'] = data['Adj Close'].pct_change()
    # data['pct_ret'] = data['pct_ret'].shift(-1)
    
    # #filter out only tradable days
    # data = data[(data['Volume'] > 4000000) & (data['Adj Close'] > 5) & (data['transaction amount'] > 40000000)]
    

    # data = data[['Volume_rel_change', 'Avg_rel_change', 'pct_ret']]

    # data.dropna(inplace=True)
    # return data



df = pd.DataFrame(columns=['Volume_rel_change', 'Avg_rel_change', 'pct_ret'])
tickers = pd.read_csv('tickers.csv')
for tick in tqdm(tickers['ticker'][:3]):
    res = feature_gen(tick)
    print(res)

# df.to_csv('dataset.csv')



NameError: name 'pd' is not defined

In [3]:
prices = pd.read_csv('dataset.csv', index_col=0)
prices.head()

,Volume_rel_change,Avg_rel_change,pct_ret
0,5.074888,0.001665,0.001479
1,5.917972,0.000996,-0.005284
2,9.849002,-0.000312,0.002328
3,4.998489,0.000983,-0.007289
4,4.387466,0.002535,0.002824
